In [0]:
data = spark.read.csv("/FileStore/tables/SMSSpamCollection", inferSchema = True, sep ='\t')

In [0]:
data.show()

+----+--------------------+
 _c0| _c1|
+----+--------------------+
 ham|Go until jurong p...|
 ham|Ok lar... Joking ...|
spam|Free entry in 2 a...|
 ham|U dun say so earl...|
 ham|Nah I don't think...|
spam|FreeMsg Hey there...|
 ham|Even my brother i...|
 ham|As per your reque...|
spam|WINNER!! As a val...|
spam|Had your mobile 1...|
 ham|I'm gonna be home...|
spam|SIX chances to wi...|
spam|URGENT! You have ...|
 ham|I've been searchi...|
 ham|I HAVE A DATE ON ...|
spam|XXXMobileMovieClu...|
 ham|Oh k...i'm watchi...|
 ham|Eh u remember how...|
 ham|Fine if thats th...|
spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows

In [0]:
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

In [0]:
data.show()

+-----+--------------------+
class| text|
+-----+--------------------+
 ham|Go until jurong p...|
 ham|Ok lar... Joking ...|
 spam|Free entry in 2 a...|
 ham|U dun say so earl...|
 ham|Nah I don't think...|
 spam|FreeMsg Hey there...|
 ham|Even my brother i...|
 ham|As per your reque...|
 spam|WINNER!! As a val...|
 spam|Had your mobile 1...|
 ham|I'm gonna be home...|
 spam|SIX chances to wi...|
 spam|URGENT! You have ...|
 ham|I've been searchi...|
 ham|I HAVE A DATE ON ...|
 spam|XXXMobileMovieClu...|
 ham|Oh k...i'm watchi...|
 ham|Eh u remember how...|
 ham|Fine if thats th...|
 spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows

In [0]:
from pyspark.sql.functions import length

In [0]:
data = data.withColumn('length', length(data['text']))

In [0]:
data.show()

+-----+--------------------+------+
class| text|length|
+-----+--------------------+------+
 ham|Go until jurong p...| 111|
 ham|Ok lar... Joking ...| 29|
 spam|Free entry in 2 a...| 155|
 ham|U dun say so earl...| 49|
 ham|Nah I don't think...| 61|
 spam|FreeMsg Hey there...| 147|
 ham|Even my brother i...| 77|
 ham|As per your reque...| 160|
 spam|WINNER!! As a val...| 157|
 spam|Had your mobile 1...| 154|
 ham|I'm gonna be home...| 109|
 spam|SIX chances to wi...| 136|
 spam|URGENT! You have ...| 155|
 ham|I've been searchi...| 196|
 ham|I HAVE A DATE ON ...| 35|
 spam|XXXMobileMovieClu...| 149|
 ham|Oh k...i'm watchi...| 26|
 ham|Eh u remember how...| 81|
 ham|Fine if thats th...| 56|
 spam|England v Macedon...| 155|
+-----+--------------------+------+
only showing top 20 rows

In [0]:
data.groupby('class').mean('length').show()

+-----+-----------------+
class| avg(length)|
+-----+-----------------+
 ham| 71.4545266210897|
 spam|138.6706827309237|
+-----+-----------------+

In [0]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [0]:
 tokenizer = Tokenizer(inputCol = 'text', outputCol = 'token_text')

In [0]:
stop_remove = StopWordsRemover(inputCol = 'token_text', outputCol ='stop_token' )

In [0]:
count_vec = CountVectorizer(inputCol ='stop_token', outputCol = 'c_vec' )

In [0]:
idf = IDF(inputCol = 'c_vec', outputCol = 'tf_idf')

In [0]:
ham_spam = StringIndexer(inputCol = 'class', outputCol= 'label')

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
clean_up = VectorAssembler(inputCols = ['tf_idf', 'length'], outputCol = 'features')

In [0]:
from pyspark.ml.classification import NaiveBayes  # classic models for NPL

In [0]:
from pyspark.ml import Pipeline

In [0]:
data_pipe = Pipeline(stages = [ham_spam, tokenizer ,stop_remove , count_vec, idf, clean_up])

In [0]:
cleaner = data_pipe.fit(data).transform(data)

In [0]:
cleaner.show()

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
class| text|length|label| token_text| stop_token| c_vec| tf_idf| features|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
 ham|Go until jurong p...| 111| 0.0|[go, until, juron...|[go, jurong, poin...|(13423,[7,11,31,6...|(13423,[7,11,31,6...|(13424,[7,11,31,6...|
 ham|Ok lar... Joking ...| 29| 0.0|[ok, lar..., joki...|[ok, lar..., joki...|(13423,[0,24,297,...|(13423,[0,24,297,...|(13424,[0,24,297,...|
 spam|Free entry in 2 a...| 155| 1.0|[free, entry, in,...|[free, entry, 2, ...|(13423,[2,13,19,3...|(13423,[2,13,19,3...|(13424,[2,13,19,3...|
 ham|U dun say so earl...| 49| 0.0|[u, dun, say, so,...|[u, dun, say, ear...|(13423,[0,70,80,1...|(13423,[0,70,80,1...|(13424,[0,70,80,1...|
 ham|Nah I don't think...| 61| 0.0|[nah, i, don't, t...|[nah, think, goes...|(13423,[36,134,31...|(13423,[36,134,31...|(13424,[36,134,31...|
 spam|FreeMsg Hey there...| 147| 1.0|[freemsg, hey, th...|[freemsg, hey, da...|(13423,[10,60,139...|(13423,[10,60,139...|(13424,[10,60,139...|
 ham|Even my brother i...| 77| 0.0|[even, my, brothe...|[even, brother, l...|(13423,[10,53,103...|(13423,[10,53,103...|(13424,[10,53,103...|
 ham|As per your reque...| 160| 0.0|[as, per, your, r...|[per, request, 'm...|(13423,[125,184,4...|(13423,[125,184,4...|(13424,[125,184,4...|
 spam|WINNER!! As a val...| 157| 1.0|[winner!!, as, a,...|[winner!!, valued...|(13423,[1,47,118,...|(13423,[1,47,118,...|(13424,[1,47,118,...|
 spam|Had your mobile 1...| 154| 1.0|[had, your, mobil...|[mobile, 11, mont...|(13423,[0,1,13,27...|(13423,[0,1,13,27...|(13424,[0,1,13,27...|
 ham|I'm gonna be home...| 109| 0.0|[i'm, gonna, be, ...|[gonna, home, soo...|(13423,[18,43,120...|(13423,[18,43,120...|(13424,[18,43,120...|
 spam|SIX chances to wi...| 136| 1.0|[six, chances, to...|[six, chances, wi...|(13423,[8,17,37,8...|(13423,[8,17,37,8...|(13424,[8,17,37,8...|
 spam|URGENT! You have ...| 155| 1.0|[urgent!, you, ha...|[urgent!, won, 1,...|(13423,[13,30,47,...|(13423,[13,30,47,...|(13424,[13,30,47,...|
 ham|I've been searchi...| 196| 0.0|[i've, been, sear...|[searching, right...|(13423,[39,96,217...|(13423,[39,96,217...|(13424,[39,96,217...|
 ham|I HAVE A DATE ON ...| 35| 0.0|[i, have, a, date...|[date, sunday, wi...|(13423,[552,1697,...|(13423,[552,1697,...|(13424,[552,1697,...|
 spam|XXXMobileMovieClu...| 149| 1.0|[xxxmobilemoviecl...|[xxxmobilemoviecl...|(13423,[30,109,11...|(13423,[30,109,11...|(13424,[30,109,11...|
 ham|Oh k...i'm watchi...| 26| 0.0|[oh, k...i'm, wat...|[oh, k...i'm, wat...|(13423,[82,214,47...|(13423,[82,214,47...|(13424,[82,214,47...|
 ham|Eh u remember how...| 81| 0.0|[eh, u, remember,...|[eh, u, remember,...|(13423,[0,2,49,13...|(13423,[0,2,49,13...|(13424,[0,2,49,13...|
 ham|Fine if thats th...| 56| 0.0|[fine, if, thats...|[fine, thats, wa...|(13423,[0,74,105,...|(13423,[0,74,105,...|(13424,[0,74,105,...|
 spam|England v Macedon...| 155| 1.0|[england, v, mace...|[england, v, mace...|(13423,[4,30,33,5...|(13423,[4,30,33,5...|(13424,[4,30,33,5...|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
train_data, test_data = cleaner.randomSplit([0.7,0.3])

In [0]:
nb = NaiveBayes()

In [0]:
spam_detector = nb.fit(train_data)

In [0]:
 test_results = spam_detector.transform(test_data)

In [0]:
 test_results.select('label', 'prediction').show()

+-----+----------+
label|prediction|
+-----+----------+
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 1.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 1.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
 0.0| 0.0|
+-----+----------+
only showing top 20 rows

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
acc_eval = MulticlassClassificationEvaluator(metricName = 'accuracy')

In [0]:
acc = acc_eval.evaluate(test_results)

In [0]:
acc

Out[46]: 0.901833234772324